In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
import pandas as pd
%load_ext autoreload
%autoreload 2

In [5]:
from scripts.proj1_helpers import *
from implementations import *
from cross_validations import *

In [6]:
y, input_data, ids = load_csv_data('../../train_nojet.csv')
y_test, test_data, ids_test = load_csv_data('../../test.csv')

In [ ]:
def build_polya(x, degree):
    """ Apply a polynomial basis to all the X features. """
    # First, we find the combinations of columns for which we have to
    # compute the product
    m, n = x.shape

    combinations = {}

    # Add combinations of same column power
    for i in range(n * degree):
        if i < n:
            combinations[i] = [i]
        else:
            col_number = i - n
            cpt = 2
            while col_number >= n:
                col_number -= n
                cpt += 1
            combinations[i] = [col_number] * cpt

    # Add combinations of products between columns
    cpt = i + 1

    for i in range(n):
        for j in range(i + 1, n):
            combinations[cpt] = [i, j]
            cpt = cpt + 1

    # Now we can fill a new matrix with the column combinations
    eval_poly = np.zeros(
        shape=(m, n + len(combinations))
    )

    for i, c in combinations.items():
        eval_poly[:, i] = x[:, c].prod(1)

    # Add square root
    for i in range(0, n):
        eval_poly[:, len(combinations) + i] = np.abs(x[:, i]) ** 0.5

    return eval_poly

In [ ]:
from itertools import product
list(map(lambda x: x[0] * x[1], product([1,2,3], repeat=2)))

In [ ]:
po = build_poly(m,3)
outpo = np.empty([po[0].shape[0], po.shape[0]*po[0].shape[1]])
print(outpo.shape)
for i in range(po[0].shape[0]):
    
    outpo[i] = np.append(po[0, i], po[1, i]) 
outpo

In [19]:
def accuracy(original_data, approximation):
    return np.equal(original_data, approximation).sum()/float(len(approximation))

In [22]:
seed = 1
degree = 4
k_fold = 4
max_iters = 50
gamma = 0.7
# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)

In [10]:
train = pd.read_csv('../../train.csv', index_col='Id', na_values=-999)


In [38]:
train

,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
Id,,,,,,,,,,,,,,,,,,,,,
100000,s,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
100001,b,160.937,68.768,103.235,48.146,NaN,NaN,NaN,3.473,2.078,...,-1.916,164.546,1,46.226,0.725,1.158,NaN,NaN,NaN,46.226
100002,b,NaN,162.172,125.953,35.635,NaN,NaN,NaN,3.148,9.336,...,-2.186,260.414,1,44.251,2.053,-2.028,NaN,NaN,NaN,44.251
100003,b,143.905,81.417,80.943,0.414,NaN,NaN,NaN,3.310,0.414,...,0.060,86.062,0,NaN,NaN,NaN,NaN,NaN,NaN,0.000
100004,b,175.864,16.915,134.805,16.405,NaN,NaN,NaN,3.891,16.405,...,-0.871,53.131,0,NaN,NaN,NaN,NaN,NaN,NaN,0.000
100005,b,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,61.619,...,2.237,282.849,3,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660
100006,s,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,2.545,...,-1.443,294.074,2,123.010,0.864,1.450,56.867,0.131,-2.767,179.877
100007,s,154.916,10.418,94.714,29.169,NaN,NaN,NaN,2.897,1.526,...,-1.761,187.299,1,30.638,-0.715,-1.724,NaN,NaN,NaN,30.638
100008,b,105.594,50.559,100.989,4.288,NaN,NaN,NaN,2.904,4.288,...,0.024,129.804,0,NaN,NaN,NaN,NaN,NaN,NaN,0.000


In [39]:
nojet = train.dropna(axis = 1, thresh=int(0.99*len(train)))

In [39]:
len(train.columns)

31

In [40]:
len(nojet.columns)

20

In [14]:
nojet.dtypes

Prediction                      object
DER_mass_transverse_met_lep    float64
DER_mass_vis                   float64
DER_pt_h                       float64
DER_deltar_tau_lep             float64
DER_pt_tot                     float64
DER_sum_pt                     float64
DER_pt_ratio_lep_tau           float64
DER_met_phi_centrality         float64
PRI_tau_pt                     float64
PRI_tau_eta                    float64
PRI_tau_phi                    float64
PRI_lep_pt                     float64
PRI_lep_eta                    float64
PRI_lep_phi                    float64
PRI_met                        float64
PRI_met_phi                    float64
PRI_met_sumet                  float64
PRI_jet_num                      int64
PRI_jet_all_pt                 float64
dtype: object

In [34]:
0.99*len(train)

247500.0

In [65]:
def normalize(df) :
    if df.name in ['Prediction', 'PRI_jet_num']:
        return df
    return (df - df.min()) / (df.max() - df.min())

In [71]:
normalized = train.fillna(0).apply(normalize, axis=0)

In [72]:
normalized.to_csv("../../train_nojet.csv")

In [73]:
normalized.head()

,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
Id,,,,,,,,,,,,,,,,,,,,,
100000,s,0.116164,0.074854,0.068128,0.009869,0.107021,0.025068,0.596501,0.521549,0.014789,...,0.455920,0.123125,2,0.060179,0.738942,0.570746,0.063846,0.637778,0.106143,0.069484
100001,b,0.135011,0.099653,0.072155,0.016983,0.000000,0.000000,0.519795,0.596238,0.000733,...,0.195099,0.075802,1,0.041252,0.580573,0.684386,0.000000,0.500000,0.500000,0.028300
100002,b,0.000000,0.235006,0.089071,0.012570,0.000000,0.000000,0.519795,0.536888,0.003293,...,0.152132,0.123969,1,0.039490,0.728162,0.177304,0.000000,0.500000,0.500000,0.027091
100003,b,0.120723,0.117983,0.055557,0.000146,0.000000,0.000000,0.519795,0.566472,0.000146,...,0.509548,0.036368,0,0.000000,0.500000,0.500080,0.000000,0.500000,0.500000,0.000000
100004,b,0.147534,0.024512,0.095662,0.005787,0.000000,0.000000,0.519795,0.672571,0.005787,...,0.361394,0.019823,0,0.000000,0.500000,0.500080,0.000000,0.500000,0.500000,0.000000


In [70]:
normalized.columns

Index(['Prediction', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h',
       'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'PRI_tau_pt',
       'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta',
       'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet', 'PRI_jet_num',
       'PRI_jet_all_pt'],
      dtype='object')

In [9]:
a= np.array([[2,3,4,54,5], [2,6,8,1,4],[2,6,8,4,4]])
np.where(a[:,3]%2==0)

(array([0, 2]),)

In [11]:
train.columns

Index(['Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'],
      dtype='object')

In [9]:
find_param_jets(y, input_data)

Current iteration=0, loss=25141.141386089774, acc=0.644564528135425, gamma=0.1, lambda=0.001
-> new average acc: 0.6170960671601671
Current iteration=0, loss=25141.141386089774, acc=0.644564528135425, gamma=0.1, lambda=0.001
Current iteration=0, loss=25141.141386089774, acc=0.6446196686057732, gamma=0.1, lambda=0.01
0.701772766121695 | d: 4 | g: 0.1 | l: 0.01 | fold 0
Current iteration=0, loss=25141.141386089774, acc=0.6496925918778087, gamma=0.1, lambda=0.01
-> new average acc: 0.7081592986228857
Current iteration=0, loss=25141.141386089774, acc=0.6467701469493535, gamma=0.1, lambda=0.6
0.7056325990460699 | d: 4 | g: 0.1 | l: 0.6 | fold 0
Current iteration=0, loss=25141.141386089774, acc=0.6520084916324336, gamma=0.1, lambda=0.6
-> new average acc: 0.7202900348758667
Current iteration=0, loss=25141.141386089774, acc=0.6449229411926883, gamma=0.1, lambda=0.1
Current iteration=0, loss=25141.141386089774, acc=0.644564528135425, gamma=0.2, lambda=0.001
Current iteration=0, loss=25141.1413

Current iteration=0, loss=25141.141386089774, acc=0.634942516059662, gamma=0.6, lambda=0.01
Current iteration=0, loss=25141.141386089774, acc=0.6349149458244879, gamma=0.6, lambda=0.6
Current iteration=0, loss=25141.141386089774, acc=0.6348322351189656, gamma=0.6, lambda=0.1
Current iteration=0, loss=25141.141386089774, acc=0.634942516059662, gamma=0.7000000000000001, lambda=0.001
Current iteration=0, loss=25141.141386089774, acc=0.634942516059662, gamma=0.7000000000000001, lambda=0.001
Current iteration=0, loss=25141.141386089774, acc=0.634942516059662, gamma=0.7000000000000001, lambda=0.01
Current iteration=0, loss=25141.141386089774, acc=0.6348873755893137, gamma=0.7000000000000001, lambda=0.6
Current iteration=0, loss=25141.141386089774, acc=0.6348322351189656, gamma=0.7000000000000001, lambda=0.1
Current iteration=0, loss=25141.141386089774, acc=0.634942516059662, gamma=0.8, lambda=0.001
Current iteration=0, loss=25141.141386089774, acc=0.634942516059662, gamma=0.8, lambda=0.001
C

KeyboardInterrupt: 

In [12]:
a=np.array([1,2,3,4,5])
np.where(a%2==0)[0]

array([1, 3])

In [3]:
a=np.array([[1,2,3],[4,5,6]])
a[a==3]=2

In [4]:
a

array([[1, 2, 2],
       [4, 5, 6]])